In [ ]:
!pip install datasets evaluate transformers[sentencepiece]

In [ ]:
from transformers import BertTokenizer, BertForTokenClassification, BertForSequenceClassification
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import TextDataset

In [ ]:
!pip install python-docx

In [ ]:
import json
# Read the JSON file
path = "/content/drive/MyDrive/model/structured_data.json"
with open(path, 'r') as f:
    dataset = json.load(f)

# Assuming the dataset is a list of dictionaries

# Define a custom Dataset class
class CustomDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

# Create an instance of CustomDataset
custom_dataset = CustomDataset(dataset)

# Create a DataLoader
batch_size = 1
train_dataloader = DataLoader(custom_dataset, batch_size=batch_size, shuffle=True)

In [ ]:
key = []
for sample in train_dataloader:
    key.extend(sample['entities'].keys())

In [ ]:
key = set(key)
print(key)
print(len(key))
num_ner_labels = len(key)

{'task', 'specific_time', 'date', 'duration', 'days', 'time'}
6


In [ ]:
l = []
for sample in train_dataloader:
    l.extend(sample['intent'])

In [ ]:
l = set(l)
print(l)
print(len(l))
num_intent_labels=len(l)

{"'Set Reminder'", "'Set Timer'", "'Set Alarm'", "'Schedule Appointment'", "'Schedule Meeting'"}
5


In [ ]:
# Load the BERT tokenizer and models for token classification and sequence classification
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
ner_model = BertForTokenClassification.from_pretrained('bert-base-uncased', num_labels=num_ner_labels)
intent_model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=num_intent_labels)

In [ ]:
ner_model

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, el

In [ ]:
intent_model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [ ]:
for sample in train_dataloader:
    print(sample['entities'])

{'task': ['cooking'], 'duration': ['1 hour']}
{'time': ['3 PM'], 'date': ['tomorrow']}
{'duration': ['10 minutes']}
{'task': ['5-minute meditation session'], 'duration': ['5 minutes']}
{'task': ['webinar'], 'time': ['2 PM'], 'duration': ['2 days']}
{'date': ['10th of next month'], 'time': ['2 PM']}
{'task': ['20-minute workout session'], 'duration': ['20 minutes']}
{'task': ['buy groceries'], 'date': ['Saturday'], 'time': ['afternoon']}
{'task': ['study session'], 'duration': ['45 minutes']}
{'time': ['7:30 AM'], 'date': ['tomorrow']}
{'task': ['water the plants'], 'time': ['9 AM'], 'days': [('Tuesday',), ('Thursday',)]}
{'task': ['pay bills'], 'date': ['last day of the month']}
{'duration': ['30 minutes'], 'task': ['call John']}
{'task': ['buy groceries'], 'date': ['tomorrow'], 'time': ['morning']}
{'date': ['end of the month'], 'time': ['4:30 PM']}
{'date': ['next Monday'], 'time': ['morning'], 'specific_time': ['10:30']}
{'date': ['March 20th'], 'time': ['10:30 AM']}
{'time': ['2:30

In [ ]:
optimizer = torch.optim.Adam([
    {'params': ner_model.parameters()},
    {'params': intent_model.parameters()}
], lr=1e-5)  # Define optimizer for both models

num_epochs = 3  # Define number of epochs

for epoch in range(num_epochs):
    for batch in train_dataloader:  # Iterate over your dataset batches
        inputs = tokenizer(batch['text'], return_tensors="pt", padding=True, truncation=True)

        # Forward pass for intent classification
        outputs_intent = intent_model(**inputs)
        intent_labels = batch['intent']
        intent_loss = torch.nn.CrossEntropyLoss()(outputs_intent.logits, intent_labels)

        # Forward pass for NER
        outputs_ner = ner_model(**inputs)
        ner_labels = batch['entities']
        ner_loss = torch.nn.CrossEntropyLoss()(outputs_ner.logits.view(-1, num_ner_labels), ner_labels.view(-1))

        # Total loss (you might adjust weights for different tasks)
        total_loss = ner_loss + intent_loss

        # Backpropagation
        total_loss.backward()
        optimizer.step()
        optimizer.zero_grad()

In [ ]:
optimizer = torch.optim.Adam([
    {'params': ner_model.parameters()},
    {'params': intent_model.parameters()}
], lr=1e-5)  # Define optimizer for both models

num_epochs = 3  # Define number of epochs

for epoch in range(num_epochs):
    for batch in train_dataloader:  # Iterate over your dataset batches
        inputs = tokenizer(batch['text'], return_tensors="pt", padding=True, truncation=True)

        # Forward pass for intent classification
        outputs_intent = intent_model(**inputs)
        intent_labels = batch['intent']
        intent_loss = torch.nn.CrossEntropyLoss()(outputs_intent.logits, intent_labels)

        # Forward pass for NER
        outputs_ner = ner_model(**inputs)
        ner_labels = batch['entities']
        ner_loss = torch.nn.CrossEntropyLoss()(outputs_ner.logits.view(-1, num_ner_labels), ner_labels.view(-1))

        # Total loss (you might adjust weights for different tasks)
        total_loss = ner_loss + intent_loss

        # Backpropagation
        total_loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        # No idea yet